In [67]:
# Run in terminal or command prompt
# python3 -m spacy download en

import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
from collections import Counter
from wordcloud import WordCloud
import seaborn as sns


In [68]:
df = pd.read_csv('EtownData.csv')
df.head()

,Questions,Answers,Category
0,When was Elizabethtown College first opened?,1899,1
1,When was Etown opened?,1899,1
2,Who originally founded Elizabethtown College?,Reverend Jacob G. Francis in 1899,1
3,Who founded Etown?,Reverend Jacob G. Francis in 1899,1
4,When was Elizabethtown College originally foun...,1899,1


In [69]:
# Convert to list
data = df.Questions.values.tolist()
# Remove Emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub(r"\'", "", sent) for sent in data]
pprint(data[:1])

['When was Elizabethtown College first opened?']


In [70]:
# Convert to list
#data = df.Questions.values.tolist()

# Remove Emails

def clean_text_round(text):
    '''
    Make text lowercase,remove text in square brackets,remove punctuation, remove words containing 
    numbers, additional punctuation and non-sensical text
    '''
 
     
    text = text.lower()
    text = re.sub('\[.*?\]', '', text) 
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[’‘’“”…]', '', text)
    text = re.sub('\n', ' ', text)
    #text = re.sub('Etown',  '',text)
    #text = re.sub('etown',  '',text)
    #text = re.sub('elizabethtown college',  '',text)

    #text = re.sub('Elizabethtown College',  '',text)

    return text




<>:14: DeprecationWarning: invalid escape sequence \[
<>:16: DeprecationWarning: invalid escape sequence \w
<>:14: DeprecationWarning: invalid escape sequence \[
<>:16: DeprecationWarning: invalid escape sequence \w
<>:14: DeprecationWarning: invalid escape sequence \[
<>:16: DeprecationWarning: invalid escape sequence \w
/tmp/ipykernel_15754/2225099262.py:14: DeprecationWarning: invalid escape sequence \[
  text = re.sub('\[.*?\]', '', text)
/tmp/ipykernel_15754/2225099262.py:16: DeprecationWarning: invalid escape sequence \w
  text = re.sub('\w*\d\w*', '', text)


In [71]:
df['Questions'] = df['Questions'].apply(clean_text_round)


In [72]:
df['Questions']

0            when was elizabethtown college first opened
1                                  when was etown opened
2           who originally founded elizabethtown college
3                                      who founded etown
4      when was elizabethtown college originally founded
                             ...                        
350    is elizabethtown college a college or a univer...
351           what year was elizabethtown college opened
352     when was elizabethtown college officially opened
353    what is the best thing elizabethtown college h...
354    what part of pennsylvania is elizabethtown col...
Name: Questions, Length: 355, dtype: object

In [73]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['when', 'was', 'elizabethtown', 'college', 'first', 'opened']]


In [74]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['college first open', 'etown open']


In [75]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [76]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  8.79641485275288 %


In [77]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=28,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=28, n_jobs=-1,
                          random_state=100)


In [78]:
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=-1,  perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

LatentDirichletAllocation(learning_method='online', n_jobs=-1, random_state=100)

In [79]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -2550.4774491713833
Perplexity:  39.659980820638665
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 28,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [80]:
# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [10, 15, 20, 25, 30]})

In [81]:
GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1,
              perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0),
       n_jobs=1,
       param_grid={'n_topics': [10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

GridSearchCV(error_score='raise',
             estimator=LatentDirichletAllocation(learning_method=None,
                                                 n_jobs=1),
             n_jobs=1,
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_topics': [10, 15, 20, 25, 30]},
             return_train_score='warn')

In [82]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.7, 'n_components': 10}
Best Log Likelihood Score:  -532.0765884710446
Model Perplexity:  19.159790706274855


In [83]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Doc0,0.030000,0.370000,0.030000,0.030000,0.370000,0.030000,0.030000,0.030000,0.030000,0.030000,1
Doc1,0.030000,0.370000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.370000,1
Doc2,0.030000,0.030000,0.030000,0.030000,0.700000,0.030000,0.030000,0.030000,0.030000,0.030000,4
Doc3,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.550000,9
Doc4,0.030000,0.030000,0.030000,0.030000,0.700000,0.030000,0.030000,0.030000,0.030000,0.030000,4
Doc5,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.550000,9
Doc6,0.030000,0.030000,0.030000,0.030000,0.240000,0.030000,0.030000,0.490000,0.030000,0.030000,7
Doc7,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.370000,0.030000,0.370000,7
Doc8,0.030000,0.030000,0.030000,0.030000,0.240000,0.030000,0.030000,0.490000,0.030000,0.030000,7
Doc9,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.370000,0.030000,0.370000,7


In [84]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,4,133
1,9,58
2,0,46
3,1,27
4,7,22
5,3,20
6,5,18
7,8,12
8,6,12
9,2,7


In [85]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/home/lip/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/lip/anaconda3/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/home/lip/anaconda3/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/home/lip/anaconda3/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:827: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will 

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
4      32.420952   8.115230       1        1  33.495020
9       0.648955  71.707100       2        1  12.640301
3      71.019684   9.971006       3        1   8.027725
5       3.785981  33.185135       4        1   7.392737
7     -42.666500  -9.222070       5        1   7.198475
1      -3.621689 -43.526516       6        1   7.141518
0      -3.632402  -4.171708       7        1   6.712152
8     -36.584400  35.804413       8        1   6.526953
2      44.391380  50.615383       9        1   5.465065
6      40.235809 -31.591665      10        1   5.400053, topic_info=             Term        Freq       Total Category  logprob  loglift
7           etown  101.000000  101.000000  Default  22.0000  22.0000
18        student   41.000000   41.000000  Default  21.0000  21.0000
3         college  123.000000  123.000000  Default  20.0000  20.0000
6   elizabethtown  111.000000  111.000000  Default  19.0000  19.0000
16         school   30.000000   30.000000  Default  18.0000  18.0000
..            ...         ...         ...      ...      ...      ...
16         school    0.154635   30.710983  Topic10  -5.4890  -2.3725
18        student    0.154635   41.408098  Topic10  -5.4890  -2.6714
7           etown    0.154641  101.745663  Topic10  -5.4889  -3.5704
6   elizabethtown    0.154640  111.322190  Topic10  -5.4889  -3.6603
3         college    0.154640  123.223922  Topic10  -5.4889  -3.7619

[242 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
0         5  0.039152            aid
0         9  0.939641            aid
1         6  0.927852        average
2         6  0.191346          class
2        10  0.765383          class
3         1  0.949491        college
3         5  0.040577        college
4         3  0.704438           cost
4         7  0.251585           cost
5         8  0.760290         credit
5        10  0.175452         credit
6         1  0.979140  elizabethtown
6         5  0.017966  elizabethtown
7         2  0.815760          etown
7         3  0.068799          etown
7         9  0.108113          etown
8         3  0.945220            fun
9         3  0.278576           good
9         7  0.668581           good
10        4  0.920428     graduation
11        4  0.929304            job
12        6  0.928601          major
13        6  0.926833           open
14        7  0.900045        program
15        4  0.964399           rate
16        8  0.976849         school
17        1  0.553848          sport
17        2  0.332309          sport
18        5  0.965995        student
19        3  0.924255          thing
20       10  0.962726           work
21        6  0.199472           year
21        7  0.748019           year, R=22, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 10, 4, 6, 8, 2, 1, 9, 3, 7])

In [86]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

/home/lip/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aid,average,class,college,cost,credit,elizabethtown,etown,fun,good,...,major,open,program,rate,school,sport,student,thing,work,year
Topic0,0.100000,0.100000,0.100000,0.100011,4.100006,0.100000,0.100009,0.100011,0.100000,10.100068,...,0.100000,0.100000,10.099994,0.100000,0.100002,0.100000,0.100004,0.100000,0.100000,12.100163
Topic1,0.100000,12.099995,4.099716,0.100009,0.100000,0.100000,0.100008,0.100009,0.100000,0.100000,...,14.099995,10.099991,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,4.099819
Topic2,15.326253,0.100000,0.100000,0.100007,0.100000,0.100000,0.100008,7.101270,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100021,0.100000,0.100000,0.100000
Topic3,0.100000,0.100000,0.100000,0.100010,12.099978,0.100000,0.100011,6.110456,10.099985,4.099915,...,0.100000,0.100000,0.100000,0.100000,0.100009,0.100000,0.100000,14.099994,0.100000,0.100000
Topic4,0.100003,0.100003,0.100000,146.999318,0.100010,0.100004,136.601290,0.100000,0.100002,0.100006,...,0.100003,0.100005,0.100003,0.100003,0.100003,6.099935,0.100003,0.100003,0.100001,0.100006


In [87]:
# Define function to predict topic for a given text document.
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization

    # Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))

    # Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)

    # Step 4: LDA Transform
    topic_probability_scores = best_lda_model.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    return topic, topic_probability_scores

# Predict the topic
mytext = ["Some text about christianity and bible"]
topic, prob_scores = predict_topic(text = mytext)
print(topic)

[0.1000000007549673, 0.10000000101806954, 0.10000000124948777, 0.100011324347025, 4.100006250842591, 0.10000000126897944, 0.10000889083246377, 0.10001081983267641, 0.1000000008470394, 10.100068280916702, 0.10000000071556982, 0.10000000073027226, 0.10000000168318095, 0.10000000129843811, 10.09999381189031, 0.10000000118343032, 0.1000022125901148, 0.10000000450999684, 0.10000441839693666, 0.1000000007381145, 0.10000000063848159, 12.100162745608156]


In [88]:
# Construct the k-means clusters
from sklearn.cluster import KMeans
clusters = KMeans(n_clusters=27, random_state=100).fit_predict(lda_output)

# Build the Singular Value Decomposition(SVD) model
svd_model = TruncatedSVD(n_components=2)  # 2 components
lda_output_svd = svd_model.fit_transform(lda_output)

# X and Y axes of the plot using SVD decomposition
x = lda_output_svd[:, 0]
y = lda_output_svd[:, 1]

# Weights for the 15 columns of lda_output, for each component
print("Component's weights: \n", np.round(svd_model.components_, 2))

# Percentage of total information in 'lda_output' explained by the two components
print("Perc of Variance Explained: \n", np.round(svd_model.explained_variance_ratio_, 2))

Component's weights: 
 [[ 0.12  0.13  0.1   0.13  0.91  0.12  0.1   0.13  0.12  0.23]
 [ 0.14  0.16  0.1   0.13 -0.36  0.16  0.13  0.15  0.13  0.85]]
Perc of Variance Explained: 
 [0.2  0.28]


In [89]:
from sklearn.metrics.pairwise import euclidean_distances

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def similar_documents(text, doc_topic_probs, documents = data, nlp=nlp, top_n=5, verbose=False):
    topic, x  = predict_topic(text)
    dists = euclidean_distances(x.reshape(1, -1), doc_topic_probs)[0]
    doc_ids = np.argsort(dists)[:top_n]
    if verbose:        
        print("Topic KeyWords: ", topic)
        print("Topic Prob Scores of text: ", np.round(x, 1))
        print("Most Similar Doc's Probs:  ", np.round(doc_topic_probs[doc_ids], 1))
    return doc_ids, np.take(documents, doc_ids)

In [90]:
df_val = pd.read_csv('ValidationEtownData.csv')
df_val.head()

,Question,Answer,Category
0,What exact year was Etown founded?,1899,1
1,What is the size of the undergraduation popula...,1688,2
2,Where can I find Etown?,Elizabethown Pennsylvania,3
3,"Is Etown Division 1, Division 2, or Division...",Devision 3,4
4,What will I have to pay to attend Etown?,"$47,170",5


In [91]:
# Get similar documents
mytext = ['What is the best thing at Etown']
doc_ids, docs = similar_documents(text=mytext, doc_topic_probs=lda_output, documents = data, top_n=1, verbose=True)
print('\n', docs[0][:500])

Topic KeyWords:  [0.10000000059674855, 0.10000000080469469, 0.1000000009874872, 0.1000101794822885, 12.0999784699297, 0.10000000100316057, 0.10001100689569317, 6.110455599321052, 10.099985343597956, 4.099915383049429, 0.10000000056555614, 0.10000000057724376, 0.1000000013303798, 0.10000000102455142, 0.10000000097785611, 0.10000000093544548, 0.10000870687143087, 0.10000000356457442, 0.10000000144872806, 14.099994024360335, 0.1000000005046365, 0.10000000146110702]
Topic Prob Scores of text:  [[0.  0.  0.  0.8 0.  0.  0.  0.  0.  0. ]]
Most Similar Doc's Probs:   [[0.  0.  0.  0.8 0.  0.  0.  0.  0.  0. ]]

 What is the best thing Etown has to offer?


In [92]:
docs[0]

'What is the best thing Etown has to offer?'

In [95]:
# Get similar documents
mytext = [df_val['Question'][0]]
doc_ids, docs = similar_documents(text=mytext, doc_topic_probs=lda_output, documents = data, top_n=1, verbose=True)
a=df.loc[df['Questions'].str.contains(docs[0])]
print('\n', 'actual', df_val["Category"][0],a.iloc[-1][2] - 1)


Topic KeyWords:  [0.1000000007549673, 0.10000000101806954, 0.10000000124948777, 0.100011324347025, 4.100006250842591, 0.10000000126897944, 0.10000889083246377, 0.10001081983267641, 0.1000000008470394, 10.100068280916702, 0.10000000071556982, 0.10000000073027226, 0.10000000168318095, 0.10000000129843811, 10.09999381189031, 0.10000000118343032, 0.1000022125901148, 0.10000000450999684, 0.10000441839693666, 0.1000000007381145, 0.10000000063848159, 12.100162745608156]
Topic Prob Scores of text:  [[0.4 0.  0.  0.  0.  0.  0.  0.  0.  0.4]]
Most Similar Doc's Probs:   [[0.4 0.  0.  0.  0.  0.  0.  0.  0.  0.4]]


IndexError: single positional indexer is out-of-bounds

/home/lip/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/lip/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/lip/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/lip/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/lip/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp mo

In [94]:
a

,Questions,Answers,Category
